In [18]:
import pandas as pd
import numpy as np
from library import start
from scipy import spatial
from statistics import mean 


clean_filepath = start.clean_filepath
table_filepath = start.table_filepath

In [2]:
docs = pd.read_csv(clean_filepath + 'text_transcripts.csv')
docs = docs.set_index('doc')
docs.sample(5)

,text,year,semester,scenario
doc,,,,
106_c_Transcript.docx,"How do you feel? Yeah. Interesting,...",2017-18,fall,feedback
11-2C.docx,How do you feel? You feel okay? Yeah. It...,2017-18,spring,behavior
2019_35_5C_Transcript.docx,"First time, yes. Yeah. So, yeah, I’m n...",2018-19,spring,behavior
75_c_Transcript.docx,"How do you feel? Good, okay. So what a...",2017-18,fall,feedback
122_c_Transcript.docx,"Yeah, and don’t be afraid, seriously, if lik...",2017-18,fall,feedback


In [3]:
docs.year.value_counts()

2017-18    144
2018-19     92
2019-20     48
Name: year, dtype: int64

In [6]:
matrix_transcripts_file = 'matrix_transcripts.csv'

matrix_transcripts = pd.read_csv(clean_filepath + matrix_transcripts_file)
matrix_transcripts = matrix_transcripts.set_index('doc')

In [7]:
# Get mean of Spring 2017-18 and Spring 2018-19

In [10]:
docs_spring18 = docs[(docs.year == '2017-18') & (docs.semester == 'spring')]
docs_spring19 = docs[(docs.year == '2018-19') & (docs.semester == 'spring')]

matrix_spring18 = matrix_transcripts[matrix_transcripts.index.isin(docs_spring18.index)]
matrix_spring19 = matrix_transcripts[matrix_transcripts.index.isin(docs_spring19.index)]

centroid_spring18 = np.mean(matrix_spring18)
centroid_spring19 = np.mean(matrix_spring19)

In [ ]:
# Mean to Mean results in a very high document similarity 

In [12]:
1 - spatial.distance.cosine(centroid_spring18, centroid_spring19)

0.9645198965239951

In [13]:
1 - spatial.distance.cosine(centroid_spring19, centroid_spring18)

0.9645198965239951

In [ ]:
# Doc to mean

In [21]:
def distance_to_mean(matrix_main, matrix_comparison):
    sims = []
    for maindoc in matrix_main.index:
        centroid = np.mean(matrix_comparison)
        matrix_comp = matrix_comparison[~matrix_comparison.index.isin([maindoc])] # exclude self
        sim = 1 - spatial.distance.cosine(matrix_main.loc[maindoc], centroid)
        sims.append(sim)    
    return sims

In [22]:
mean(distance_to_mean(matrix_spring18, matrix_spring19))

0.8754470792464875

In [23]:
mean(distance_to_mean(matrix_spring19, matrix_spring18))

0.887391234917196

In [24]:
# Pairwise

In [29]:
sims = []
for maindoc in matrix_spring18.index:
    for compdoc in matrix_spring19.index:
        sim = 1 - spatial.distance.cosine(matrix_spring18.loc[maindoc], matrix_spring19.loc[compdoc])
        sims.append(sim)
mean(sims)

0.8054762823274201

In [30]:
sims = []
for maindoc in matrix_spring19.index:
    for compdoc in matrix_spring18.index:
        sim = 1 - spatial.distance.cosine(matrix_spring19.loc[maindoc], matrix_spring18.loc[compdoc])
        sims.append(sim)
mean(sims)

0.8054762823274201

## I think that the docs to mean version is different depending on the study that is the baseline because it depends on the variability in the study. If this is true, then spring 2019 should have a bigger session similarity measure than spring 2018.

In [33]:
mean(distance_to_mean(matrix_spring18, matrix_spring18))

0.908180980385631

In [34]:
mean(distance_to_mean(matrix_spring19, matrix_spring19))

0.9197988754958992

### we see that.